In [1]:
import matplotlib.pyplot as plt
import numpy as np
from glidertest import fetchers
from glidertest import tools

ModuleNotFoundError: No module named 'glidertest'

In [2]:
cd

C:\Users\u241346


C:\Users\u241346\AppData\Local\anaconda3\envs\chiara_base\Lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
C:\Users\u241346\AppData\Local\anaconda3\envs\chiara_base\Lib\site-packages\IPython\core\magics\osm.py:428: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Load dataset

Load an example dataset using `glidertest.fetchers.load_sample_dataset`

Alternatively, use your own with e.g. `ds = xr.open_dataset('/path/to/yourfile.nc')`

In [ ]:
ds = fetchers.load_sample_dataset()

In [ ]:
ds

### Check basic data and water column structure first

In [ ]:
tools.plot_basic_vars(ds,v_res=1, start_prof=1, end_prof=int(ds.PROFILE_NUMBER.max()))

### Check for up-downcast bias in Salinity, Temperature, Chlorophyll and Oxygen

In [ ]:
fig, ax = plt.subplots(1,4,figsize=(15, 5))
tools.test_updown_bias(ds, ax[0],var='PSAL',v_res=0.1)
tools.test_updown_bias(ds,ax[1], var='CHLA',v_res=0.1)
tools.test_updown_bias(ds, ax[2], var='TEMP',v_res=0.1)
tools.test_updown_bias(ds, ax[3], var='DOXY',v_res=0.1)

### Chlorophyll

*  Check bottom data and see if we have stable data that can be used for calibration. We also check stability of data to assess whether or not we have suspicious drift over the mission
* We then check if data is affected by non photochemical quenching (NPQ). NPQ is a physiological response to high light environments used by plants and algae to protect themselves from damage and causes an evident weakening in fluorescence signal during the day. With the 'test_npq' function, we plot a selected section of chlorophyll data to see if any NPQ effect in the top few meters is visible and then we plot a selcted day daily and night average to check again any NPQ effect.

(Reminder this tests mission had issues with FLBBCD as it stopped working few days into the missiona and got flooded)

In [ ]:
tools.chl_first_check(ds)

In [ ]:
# We need to make sure we time as an index. If data already has time as an index, this cell will produce an error and can be ignored 
ds = ds.set_xindex('TIME')

In [ ]:
tools.check_npq(ds, offset = np.timedelta64(1, "h"), start_time = '2023-09-06', end_time = '2023-09-10', sel_day=2)

#### Non photochemical quenching corrections
NPQ occurs only during the daytime, therefore when night measurements close in time and 
space are available, they can be used to correct daytime profiles. Different methods exist for NPQ 
correction. We recommend the paper by Thomalla et al. (2018) for a good overview on all possible methods. 
GliderTools (https://glidertools.readthedocs.io/en/latest/optics.html#quenching-correction) provides good tools for correction

Thomalla, S. J., W. Moutier, T. J. Ryan-Keogh, L. Gregor, and J. Schütt. 2018. An optimized 
method for correcting fluorescence quenching using optical backscattering on autonomous 
platforms. Limnology and Oceanography: Methods, 16: 132-144. DOI: 
https://doi.org/10.1002/lom3.10234